## Установим нужные библиотеки для инициализации модели

In [1]:
!pip install langchain langchain_community faiss-gpu --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 1.5 MB/s eta 0:00:00


In [2]:
!pip install loguru

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.4 MB/s eta 0:00:00


In [3]:
%pip install --quiet transformers==4.34.1 accelerate==0.24.0 sentencepiece==0.1.99 optimum==1.13.2 peft==0.5.0 bitsandbytes==0.41.2.post2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 23.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1

In [4]:
!gdown "12CmKXfBqhAAsuJObCphZCVS3JX9sUASN"

Downloading...
From: https://drive.google.com/uc?id=12CmKXfBqhAAsuJObCphZCVS3JX9sUASN
To: /content/processed_Инструкция D-1C1-1.10.14  Учет НДС_v7.csv
100% 371k/371k [00:00<00:00, 98.2MB/s]


## Импортируем файлы и заполняем датасет

In [5]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
import warnings
from typing import List

import numpy as np
import torch
from langchain.docstore.document import Document as LangchainDocument
from langchain_community.llms import HuggingFacePipeline
from langchain.pydantic_v1 import BaseModel
from langchain.schema.embeddings import Embeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from loguru import logger
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers import AutoModel, AutoTokenizer, DataCollatorForSeq2Seq, pipeline, \
    AutoModelForCausalLM, T5ForConditionalGeneration

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Благодаря коду ниже мы объединяем данные из всех файлов в 1 датасет. При необходимости возможно обучить модель отвечать как на вопросы по конкретной документации, так и на вопросы по любому виду документации

In [6]:
path = '/content/drive/MyDrive/AtomicHack/Processed Files'
files = os.listdir(path)

Если достаточно обучения на одном датасете, то выбираем код, находящийся ниже

In [7]:
# обучение на одном датасете
with open(path+'/'+'processed_Инструкция D-1C1-1.10.14  Учет НДС_v7.csv', encoding='utf-8') as f:
  dataset = [line.strip() for line in f.readlines()]

Если нужно использовать все данные, то используем нижеследующий код

In [ ]:
# обучение на всех доступных данных
dataset = list()
for file in files:
  with open(path+'/'+file, encoding='utf-8') as f:
    for line in f.readlines():
      dataset.append(line.strip())

Инициализируем модель и указываем промпты

In [8]:
PROMPT_IN_CHAT_FORMAT_1 = [
    {
        "role": "system",
        "content": """Используя информацию, содержащуюся в контексте,
дайте исчерпывающий ответ на вопрос.
Отвечайте только на заданный вопрос, ответ должен быть кратким и релевантным.
При необходимости укажите номер источника документа.
Если ответ невозможно вывести из контекста, не давайте ответа.""",
    },
    {
        "role": "user",
        "content": """Контекст:
{context}
---
Вот вопрос, на который вам нужно ответить.

Вопрос: {question}""",
    },
]

PROMPT_IN_CHAT_FORMAT_2 = [
    {
        "role": "system",
        "content": """Ты - высокоинтеллектуальная языковая модель, созданная для предоставления исчерпывающих и релевантных ответов на вопросы с использованием предоставленного контекста.
Твоя задача - внимательно изучить предоставленный контекст и затем дать максимально полный и информативный ответ на вопрос, основываясь исключительно на этом контексте.
Если ответ невозможно вывести из контекста, честно признай это.
При ответе используй разделы, четкую структуру и, по возможности, нумерацию пунктов для лучшей читаемости.
Цитируй части контекста, сопровождая их указанием номера источника. Избегай общих фраз и постарайся дать конкретную, основанную на фактах информацию.
 Твой ответ должен быть исчерпывающим, но при этом лаконичным и по существу.""",
    },
    {
        "role": "user",
        "content": """Контекст:
{context}

Вопрос:
{question}""",
    },
]


def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()


class OneCTextsEmbedder(BaseModel, Embeddings, extra='allow'):
    def __init__(self, model_name='cointegrated/rubert-tiny2', device_type='cuda', pbar=True, *args,
                 **kwargs):
        super().__init__(*args, **kwargs)
        self.device = torch.device(device_type)

        self.model = AutoModel.from_pretrained(model_name).to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        self.data_collator = DataCollatorForSeq2Seq(tokenizer=self.tokenizer, model=model_name)
        self.pbar = pbar

    def _inference(self, texts: List[str]) -> List[List[float]]:
        model_inputs = [self.tokenizer(text, max_length=256, padding='max_length',
                                       truncation=True, return_tensors='pt')
                        for text in texts]
        loader = DataLoader(model_inputs, batch_size=32, shuffle=False,
                            collate_fn=self.data_collator)
        embeddings = []
        batches = loader if not self.pbar else tqdm(loader, total=len(loader),
                                                    desc='Iterating over documents')
        for batch in batches:
            batch['input_ids'] = batch['input_ids'].squeeze(1)
            batch['attention_mask'] = batch['attention_mask'].squeeze(1)
            batch['token_type_ids'] = batch['token_type_ids'].squeeze(1)
            batch.to(self.device)
            with torch.no_grad():
                model_output = self.model(**batch)
                emb = model_output.last_hidden_state[:, 0, :]
                emb = torch.nn.functional.normalize(emb).cpu()
                embeddings.append(emb.float().numpy())
        embeddings = np.concatenate(embeddings, 0).reshape(-1, embeddings[0].shape[-1])
        return embeddings.tolist()

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self._inference(texts)

    def embed_query(self, text: str) -> List[float]:
        return self._inference([text])[0]


def construct_pipeline(reader_name, device_type='cuda', is_interactive=True):
    embedder = OneCTextsEmbedder(device_type=device_type, pbar=is_interactive)
    logger.info('Создана модель-эмбеддер')
    code_chunks = [LangchainDocument(sample) for sample in dataset]
    logger.info('Строим индекс на документах')
    db = FAISS.from_documents(code_chunks, embedder, distance_strategy=DistanceStrategy.COSINE)

    model = AutoModelForCausalLM.from_pretrained(
        reader_name, device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
        load_in_8bit=True, torch_dtype=torch.float32, llm_int8_enable_fp32_cpu_offload=True
        # веса в 8-bit; layernorms и activations в fp32
    )
    # model = T5ForConditionalGeneration('cointegrated/rut5-base-multitask')
    tokenizer = AutoTokenizer.from_pretrained(reader_name, trust_remote_code=True)

    model.eval()

    text_generator = pipeline(
        task="text-generation",
        model=model,
        tokenizer=tokenizer,
        do_sample=True,
        temperature=0.2,
        repetition_penalty=1.1,
        return_full_text=False,
        max_new_tokens=512,
    )

    reader_llm = HuggingFacePipeline(
        pipeline=text_generator,
        model_kwargs={"temperature": 0.5, "max_length": 512, "device": "cuda"},
    )
    logger.info('Создана LLM-читатель')

    rag_prompt = tokenizer.apply_chat_template(
        PROMPT_IN_CHAT_FORMAT_1, tokenize=False, add_generation_prompt=True
    )
    logger.info('RAG пайплайн готов')
    return reader_llm, db, rag_prompt


def rag_inference(reader_llm, vector_db, rag_prompt, query):
    retrieved_docs = vector_db.similarity_search(query=query, k=5)
    retrieved_docs_text = [doc.page_content for doc in retrieved_docs]
    context = "\nПолученные документы:\n"
    context += "".join(
        [f"Документ {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)])

    final_prompt = rag_prompt.format(question=query, context=context)

    answer = reader_llm(final_prompt)
    return answer


def rag_interactive_inference(reader_name="IlyaGusev/saiga_llama3_8b",  #
                              device_type="cuda"):
    reader_llm, vector_db, rag_prompt = construct_pipeline(reader_name=reader_name,
                                                           device_type=device_type)

    while True:
        user_query = input("Введите вопрос. Чтобы закончить диалог, введите <<END>>: ")
        if user_query == '<<END>>':
            break
        rag_answer = rag_inference(reader_llm, vector_db, rag_prompt, user_query)
        print("Ответ модели: ")
        print(rag_answer)
        print('\n', '=' * 45)

In [9]:
import warnings

warnings.filterwarnings('ignore')

In [10]:
rag_interactive_inference()

2024-06-16 01:00:44.588 | INFO     | __main__:construct_pipeline:94 - Создана модель-эмбеддер
2024-06-16 01:00:44.592 | INFO     | __main__:construct_pipeline:96 - Строим индекс на документах


Iterating over documents:   0%|          | 0/4 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2024-06-16 01:04:11.016 | INFO     | __main__:construct_pipeline:124 - Создана LLM-читатель
2024-06-16 01:04:11.085 | INFO     | __main__:construct_pipeline:129 - RAG пайплайн готов


Введите вопрос. Чтобы закончить диалог, введите <<END>>: что делать если не заходит в систему


Iterating over documents:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Ответ модели: 
Если не заходит в систему, то следует проверить следующие пункты:

1. **Убедитесь, что вы правильно ввели логин и пароль**. Проверьте, что введенные данные соответствуют действительным данным пользователя.

2. **Обновите браузер или очистите кэш**. Иногда проблемы могут возникнуть из-за устаревшего браузера или накопленных данных в кэше. Попробуйте использовать другой браузер или очистить историю и cookies текущего браузера.

3. **Проверьте подключение к интернету**. Убедитесь, что ваш компьютер подключен к интернету и скорость соединения достаточна для работы с системой.

4. **Свяжитесь с администратором системы**. Если проблема сохраняется, возможно, есть технические проблемы со стороны сервера или конфигурации системы. Свяжитесь с администратором системы или поддержкой разработчика программного обеспечения для получения дополнительной помощи.

5. **Перезагрузите компьютер**. Иногда простая перезагрузка может решить проблему.

6. **Проверьте наличие обновлений**. Убеди